# Data Analysis

Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ in the format

![img](./squeme.svg)

Example of entry

In [2]:
from IPython.display import JSON 

In [10]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [13]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [1]:
import pandas as pd

In [2]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


In [3]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [6]:
import itertools

In [605]:
def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
    '''
    sltns=[]
    #p = [ (x,y) for x in l for y in l if x<=y]
    p = list(itertools.combinations_with_replacement(l,2))
    Ss = [abs(sum(x)) for x in p]
    # capture all the possible scalar, Ss, values
    d = dict(zip(p,Ss))
    
    # Recover list, ll, from proper scalar, s and compare with input, l
    for s in set(Ss):
        tmp=[k for k, v in d.items() if v==s]
        # flatten the list
        ll=[item for sublist in tmp for item in sublist]
        if set(l)==set(ll):
            sltn={}
            sltn['S']=s
            sltn['ψ']=[k for k, v in d.items() if v==s]
            sltns.append(sltn)

    # Quality check 
    for sltn in sltns:
        s=sltn['S']
        # remove good repeated, r
        rptd=set([x for x in l if l.count(x)>1])
        for r in rptd.copy():
            v=(r,r)
            if abs(sum(v))==s and v in d.keys():
                rptd.remove(r)
        
        # Combine repeateds
        cmbs=list(itertools.combinations(rptd,2))
        # Orphan repetead
        if rptd and not cmbs:
            return []
        
        #bad combination
        for v in cmbs:
            if not (sum(v)==s and l.count(v[0])==l.count(v[1])):
                return []            
    
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]

In [607]:
df['hidden']=df['solution'].apply(get_hidden_sector)

In [611]:
df[df['hidden'].apply(len)>0][['solution'],[]

,l,k,solution,gcd,n,hidden
199,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,"[{'S': 5, 'ψ': [(3, -8), (5, -10), (9, -14), (..."
200,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,"[{'S': 5, 'ψ': [(5, -10), (6, -11), (7, -12), ..."
916,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,"[{'S': 4, 'ψ': [(1, -5), (2, 2), (2, -6), (-3,..."
924,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,8,"[{'S': 1, 'ψ': [(3, -4), (-6, 7), (8, -9)]}]"
938,"[1, -1, 1]","[-2, -1, -2]","[1, -2, -7, -7, 8, 8, 10, -11]",1,8,"[{'S': 1, 'ψ': [(1, -2), (-7, 8), (10, -11)]}]"
...,...,...,...,...,...,...
389820,"[-2, 3, -2, 1, -2]","[-1, -3, 0, 1, -2]","[2, 3, 5, 6, 8, 12, 13, -14, -20, -21, -26, 32]",6,12,"[{'S': 18, 'ψ': [(2, -20), (3, -21), (5, 13), ..."
389898,"[2, 1, 3, 1, 3]","[-1, -2, -1, -3, 1]","[1, 3, 4, 8, 10, 11, 12, -15, -18, -22, -26, 32]",4,12,"[{'S': 14, 'ψ': [(1, -15), (3, 11), (4, 10), (..."
389902,"[2, -1, -3, -1, 2]","[-1, -2, -1, -2, 3]","[3, 4, 6, 9, -13, -13, 17, -18, -19, 28, 28, -32]",3,12,"[{'S': 15, 'ψ': [(3, -18), (4, -19), (6, 9), (..."
390045,"[-3, 1, -2, 1, -2]","[-1, 3, -2, -1, -2]","[4, -9, 12, -13, -13, 17, -19, 24, -27, 28, 28...",6,12,"[{'S': 15, 'ψ': [(4, -19), (-9, 24), (12, -27)..."


In [273]:
sltn=[]
i=0
l=[1,2,-3,4,5]
ll=l.copy()
CONTINUE=True

In [274]:
while CONTINUE:
    x=[(ll[0],ll[0])]+list(itertools.product([ll[0]],ll[1:]))
    if i==0:
        xs=[abs(sum(y)) for y in x]
        xd=dict(zip(xs,x))
        print(x,xs,xd,ll[0])
        tmp=dict(zip(ll,x))
        ll=[xx for xx in ll if xx!=ll[0]] #change for function
        print(tmp,xd)
    if i>0:
        xs=set(xs).intersection({abs(sum(y)) for y in x})
        xd=dict([(k,xd[k]) for k in xs])
        print(x,xd,xs,ll[0])
        for kk in xs:
            print(kk)
            sltn.append(xd[kk])
            ll=[xx for xx in ll if xx!=ll[0] and xx not in xd[kk]] #change for function
            # update xd
            xd=dict([(kk,y) for y in x if abs(sum(y))==kk ])
            if not ll:
                sltn.append(xd[kk])
                CONTINUE=False
    #ll=[xx for xx in ll if xx!=ll[0]] #change for function
    print(x,ll,sltn,xd)
    i+=1
    print(i)
    #Emergency exit

[(1, 1), (1, 2), (1, -3), (1, 4), (1, 5)] [2, 3, 2, 5, 6] {2: (1, -3), 3: (1, 2), 5: (1, 4), 6: (1, 5)} 1
{1: (1, 1), 2: (1, 2), -3: (1, -3), 4: (1, 4), 5: (1, 5)} {2: (1, -3), 3: (1, 2), 5: (1, 4), 6: (1, 5)}
[(1, 1), (1, 2), (1, -3), (1, 4), (1, 5)] [2, -3, 4, 5] [] {2: (1, -3), 3: (1, 2), 5: (1, 4), 6: (1, 5)}
1
[(2, 2), (2, -3), (2, 4), (2, 5)] {6: (1, 5)} {6} 2
6
[(2, 2), (2, -3), (2, 4), (2, 5)] [-3, 4] [(1, 5)] {6: (2, 4)}
2
[(-3, -3), (-3, 4)] {6: (2, 4)} {6} -3
6
[(-3, -3), (-3, 4)] [] [(1, 5), (2, 4), (-3, -3)] {6: (-3, -3)}
3


In [275]:
sltn

[(1, 5), (2, 4), (-3, -3)]

In [265]:
i=1
x=[(ll[0],ll[0])]+list(itertools.product([ll[0]],ll[1:]))
if i==0:
    xs=[abs(sum(y)) for y in x]
    xd=dict(zip(xs,x))
    print(x,xs,xd,ll[0])
    tmp=dict(zip(ll,x))
    ll=[xx for xx in ll if xx!=ll[0]] #change for function
    print(tmp,xd)
if i>0:
    xs=set(xs).intersection({abs(sum(y)) for y in x})
    xd=dict([(k,xd[k]) for k in xs])
    print(x,xd,xs,ll[0])
    for kk in xs:
        print(kk)
        sltn.append(xd[kk])
        ll=[xx for xx in ll if xx!=ll[0] and xx not in xd[kk]] #change for function
        # update xd
        xd=dict([(kk,y) for y in x if abs(sum(y))==kk ])
        if not ll:
            sltn.append(xd[kk])        
#ll=[xx for xx in ll if xx!=ll[0]] #change for function
print(x,ll,sltn,xd)    

[(-3, -3), (-3, 4)] {6: (2, 4)} {6} -3
6
[(-3, -3), (-3, 4)] [] [(1, 5), (2, 4), (-3, -3)] {6: (-3, -3)}


In [259]:
i=1
x=[(ll[0],ll[0])]+list(itertools.product([ll[0]],ll[1:]))
if i>0:
    xs=set(xs).intersection({abs(sum(y)) for y in x})
    xd=dict([(k,xd[k]) for k in xs])
    print(x,xd,xs,ll[0])
    for kk in xs:
        print(kk)
        sltn.append(xd[kk])
        ll=[xx for xx in ll if xx!=ll[0] and xx not in xd[kk]] #change for function
        # update xd
        xd=dict([(kk,y) for y in x if abs(sum(y))==kk ])
        if not ll:
            sltn.append(xd[kk])       
#ll=[xx for xx in ll if xx!=ll[0]] #change for function
print(x,ll,sltn,xd)


[(-3, -3), (-3, 4)] {6: (2, 4)} {6} -3
6
[(-3, -3), (-3, 4)] [] [(1, 5), (2, 4), (-3, -3)] {6: (-3, -3)}


In [246]:
i=2
x=[(ll[0],ll[0])]+list(itertools.product([ll[0]],ll[1:]))
if i>0:
    xs=set(xs).intersection({abs(sum(y)) for y in x})
    xd=dict([(k,xd[k]) for k in xs])
    print(x,xd,xs,ll[0])
    for kk in xs:
        print(kk)
        sltn.append(xd[kk])
        ll=[xx for xx in ll if xx!=ll[0] and xx not in xd[kk]] #change for function
        # update xd
        xd=dict([(kk,y) for y in x if abs(sum(y))==kk ])
#ll=[xx for xx in ll if xx!=ll[0]] #change for function
print(x,ll,sltn,xd)

IndexError: list index out of range

In [65]:
for y in x[0:n]:
    sumy=sum(y)
    for yy in  ,x[n:n*2-1]

2
3
-2
5
6
